In [1]:
from src.constants import *
from src.model.my_model import MyModel
from src.model.baselines import UNet
import numpy as np
from torch.profiler import profile, record_function, ProfilerActivity
from src.model.blocks import *
import torch.nn as nn

In [2]:
example2 = torch.rand(size=[2, 1, 128, 128, 128]).cuda()
mask = torch.rand(size=[2, 3, 128, 128, 128]).cuda()
loss_fn = monai.losses.DiceCELoss()

In [3]:
model = MyModel(in_channels=1,
                 out_channels=3,
                 lower_channels=32,
                 big_channel=32,
                 patch_size=8,
                 embed_dim=256,
                 skip_transformer=False).cuda()
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-4)

# model = UNet.cuda()

In [5]:
%%time
out = model(example2)
l = loss_fn(out, mask)
l.backward()
optimizer.step()

CPU times: user 267 ms, sys: 366 ms, total: 633 ms
Wall time: 629 ms


In [3]:
torch.cuda.empty_cache()

In [2]:
model = MyModel(in_channels=1,
                 out_channels=3,
                 lower_channels=32,
                 big_channel=32,
                 patch_size=8,
                 embed_dim=256,
                 skip_transformer=True).cuda()

# model = UNet.cuda()
# model = SingleConvBlock(in_channels=1, out_channels=32, padding='same')
model = nn.Conv3d(in_channels=1, out_channels=32, padding='same', kernel_size=3, stride=1)

relu = nn.PReLU().cuda()
d = nn.Dropout(p=0.1, inplace=True)
norm = nn.InstanceNorm3d(32)



model = model.cuda()

inputs = torch.rand(size=[4, 1, 128, 128, 128]).cuda()

with profile(activities=[ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        profile_memory=True, record_shapes=True, with_stack=True) as prof:
    o = model(inputs)
    o = relu(o)
    # o = d(o)
    o = norm(o)

print(prof.key_averages(group_by_input_shape=True).table(sort_by="cuda_memory_usage"))

STAGE:2023-04-26 11:49:37 2915692:2915692 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls                                                                      Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------------------------------------------------------------------

STAGE:2023-04-26 11:49:38 2915692:2915692 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-04-26 11:49:38 2915692:2915692 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
